In [ ]:
#H.9.1a
k_bits = 2
main_bits = 2
all_bits = k_bits + main_bits

k_wires = range(k_bits)
main_wires = range(k_bits, all_bits)
dev = qml.device("default.qubit", wires=all_bits)  

def walk(alpha_list, U_list):
    SELECT(U_list)
    mat = PREPARE_matrix(alpha_list)
    r = np.zeros(shape=mat.shape)
    for _ in range(len(r)):
        if _ == 0:
            r[_][_] = 1.0
            continue
        r[_][_] = -1.0
    R = mat.conj().T @ r @ mat
    qml.QubitUnitary(R, wires=k_wires)

In [ ]:
#H.9.1b
@qml.qnode(dev)
def walk_circuit(alpha_list, U_list, steps):
    qml.QubitUnitary(PREPARE_matrix(alpha_list), wires=k_wires)
    for _ in range(steps):
        walk(alpha_list,U_list)
    return qml.state()

In [ ]:
#H.9.2a
targ_bits = k_bits + main_bits
targ_wires = range(targ_bits)
k_wires = range(k_bits)

def eigenstate_prep(alpha_list, U_list, E):
    alpha = sum(alpha_list) 
    c_plus =  np.sqrt(2 * (1 + E/alpha))**(-1)
    c_minus = np.sqrt(2 * (1 - E/alpha))**(-1)
    qml.QubitUnitary(PREPARE_matrix(alpha_list), wires= k_wires)
    mat = SELECT_matrix(U_list)
    mat = 1/np.sqrt(2) * ((c_plus + 1j * c_minus) * np.eye(2**targ_bits) +  (c_plus - 1j * c_minus) * mat)
    qml.QubitUnitary(mat, wires=targ_wires)
    c = [c_plus, c_minus] 

In [ ]:
#H.9.2b
p_bits = 8
p_wires = range(targ_bits, targ_bits + p_bits) 

dev2 = qml.device("default.qubit", wires=targ_bits + p_bits)

@qml.qnode(dev2)
def qpe_circuit(alpha_list, U_list, state, E):
    eigenstate_prep(alpha_list,U_list,E)
    qml.QuantumPhaseEstimation(walk_matrix(alpha_list,U_list),targ_wires,p_wires)
    return qml.probs(wires=p_wires)

In [ ]:
#H.9.2c
U_list = [np.kron(qml.Hadamard.compute_matrix(), qml.Hadamard.compute_matrix()),
        np.kron(-qml.Hadamard.compute_matrix(), qml.PauliX.compute_matrix()),
        np.kron(-qml.PauliX.compute_matrix(), qml.Hadamard.compute_matrix()),
        np.kron(qml.PauliX.compute_matrix(), qml.PauliX.compute_matrix())]
alpha_list = [2.0, np.sqrt(2), np.sqrt(2),1.0] # MODIFY THIS

phase_estimated_00 = qpe_circuit(alpha_list, U_list,[0,0],1.0)
phase_estimated_01 = qpe_circuit(alpha_list, U_list,[0,1],-1.0)

phase_estimated_00 = phase_estimated = np.argmax(phase_estimated_00) / 2 ** p_bits
phase_estimated_01 = phase_estimated = np.argmax(phase_estimated_01) / 2 ** p_bits


alpha = sum(alpha_list)

energy_estimated_00 = alpha*np.cos(2*np.pi*phase_estimated_00)
energy_estimated_01 = alpha*np.cos(2*np.pi*phase_estimated_01)

print("The estimated energy for state |00> is", energy_estimated_00, ".")
print("The estimated energy for state |01> is", energy_estimated_01, ".")

In [ ]:
#H.9.3a
def S(time, alpha):
    hbar = 1e-34
    output = np.eye(2**(targ_bits + p_bits), dtype = 'complex_')
    for k in range(2**targ_bits):
        for i in range(2**p_bits):
            index = 2**p_bits*k + i
            val = - time * alpha / hbar * np.cos(2 * np.pi * 1/(2**p_bits) * index)
            output[index, index] = np.exp(1j * val)
    return output

In [ ]:
#H.9.3b
@qml.qnode(dev2)
def qubitization(alpha_list, U_list, time):
    """Perform Hamiltonian simulation using a simplified qubitization circuit.
    
    Args:
        alpha_list (array[float]): A list of coefficients.
        U_list (list[array[complex]]): A list of unitary matrices, stored as complex arrays.
        time (float): The time to evolve the Hamiltonian for.
    """
    prep = PREPARE_matrix(alpha_list)
    prep_dagger = np.conjugate(np.transpose(prep))
    alpha = sum(alpha_list)
    
    ##################
    # YOUR CODE HERE #
    ##################
    qml.QubitUnitary(prep,wires = [_ for _ in range(k_bits)])
    qml.QuantumPhaseEstimation(walk_matrix(alpha_list,U_list),targ_wires, p_wires)
    qml.QubitUnitary(S(time, alpha), wires = [_ for _ in range(10)])
    qml.adjoint(qml.QuantumPhaseEstimation)(walk_matrix(alpha_list,U_list),targ_wires,p_wires)
    qml.QubitUnitary(prep_dagger, wires = [_ for _ in range(k_bits)])
    
    return qml.probs(wires=range(k_bits, k_bits + main_bits))
